# Mount your drive (in case you want to save some results) + GitHub directory
* Please use GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Git
!git clone https://github.com/Hesam-92-19/Transfer_Learning_SHM



Mounted at /content/drive
Cloning into 'Transfer_Learning_SHM'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), 1.74 MiB | 4.67 MiB/s, done.


# Download + Unzip data files

In [ ]:
# Creating the ZIP file 
import shutil
from zipfile import ZipFile
import urllib
AD1='https://drive.google.com/uc?export=download&id=1-422on4tCy7EbCvKLdMboy9y_X0d703E&confirm=t&uuid=b1f020cb-df7a-46fb-ad9f-0de7c901dc97&at=ALgDtsxV0uvcfl6acmBvQTEYcQGW:1678645176383'
AD2='https://drive.google.com/uc?export=download&id=1XeU0hfYAvIR6woDqRp1EGt-4YdQmAIFW&confirm=t&uuid=b1f020cb-df7a-46fb-ad9f-0de7c901dc97&at=ALgDtsxV0uvcfl6acmBvQTEYcQGW:1678645176383'
urllib.request.urlretrieve(AD1, 'A.zip')
urllib.request.urlretrieve(AD2, 'B.zip')

## Extracting the ZIP files
with ZipFile('A.zip', 'r') as f:
  f.extractall('Detectors')
with ZipFile('B.zip', 'r') as f:
  f.extractall('Dataset')

import os
os.remove('A.zip')
os.remove('B.zip')

#Utils Notebook which is called in this cell contains all the functions needed to carry out the TL.

In [ ]:
import sys
Folder_N='Test'
path_M = "/content/"
sys.path.append(path_M)
%cd /content/Transfer_Learning_SHM
%run Utils.ipynb
%cd

## Select your precision level (Float 10^-40 (max S=40), Double 10^-80 (max S=80))
#Prec='Float'
#Precision=40
Prec='Double'
Precision=80
torch.set_default_tensor_type('torch.cuda.'+Prec+'Tensor')



# Source Detector
* Note that Z24 has twice N and half W compared to QUGS and Yellow frame. But for index compatibility in writing codes here, for instance, if Z24 has $W=1000$ ans $N=30$, it is labeled as $W$=2000 and $N=15$. Recall that for the transfer all is needed is the same W*N

In [ ]:
SourceK()
NK()
WK()

interactive(children=(Dropdown(description='Source Class', options=('Yellow', 'QUGS', 'Z24'), style=Descriptio…

interactive(children=(Dropdown(description='Source N', options=('15', '10', '6'), style=DescriptionStyle(descr…

interactive(children=(Dropdown(description='Source W', options=('2000', '1000', '500'), style=DescriptionStyle…

# Source Detector
* Load Source Detector and its Spectrum


In [ ]:
exec('D = CustomNeuralNetworkS%s().to(device)'%(str(N)))
checkpoint =torch.load(path_M+'Detectors/'+Source[0]+'D1S'+str(NS_D)+'_'+str(WS_D)+'.pth')
D.load_state_dict(checkpoint['D'].state_dict())
Spec=checkpoint['Spec']


# Select Taget
* If you select the Traget as Source, then the Source Detector performance on it domain is later plotted

In [ ]:
SourceKT()

interactive(children=(Dropdown(description='Target Class', options=('Yellow', 'QUGS', 'Z24'), style=Descriptio…

# Load Target Data
By selecting the Traget the following data is imported:
* DM=features
* CLass_CLT= Class sizes (number of data points)
* E1T, ZopT = Cummulative Class_CLT for plot purposes

In [ ]:
[DMT, Data_CLT, E1T, ZopT]=Load_Data(Target,NS_D,WS_D,path_M)
[DMS, ZetS, E1S, ZopS]=Load_Data(Source,NS_D,WS_D,path_M)

#Transfer Learning-Binary SDD

In [ ]:
def Binary_SDD(DMT,WS_D,NS_D,E1T,Data_CLT,D,Spec,Precision,TL=True):
  CL=len(E1T)-1
  PP=np.zeros((CL,1))
  RR=np.zeros((CL,1))
  FF=np.zeros((CL,1))
  E1=np.copy(E1T)
  for Class in range(1,CL):
    DM=DMT
    W=WS_D
    S=NS_D
    Zet=np.copy(Data_CLT)
    E1Q=np.copy(E1T)
    Source_Spectrum=Spec
    Precision=80
    TL="True"
    Plot=False
    if Target.find("Z24")==-1:
      Coef=0.1
    else:
      Coef=0.5
    Coef2=0.4
    D_initiation=np.array([E1,np.zeros((len(E1)))])
    if TL=="False":
      Zet=np.array([0,int(E1Q[1])-int(E1Q[1]*Coef),E1Q[Class+1]-E1Q[Class]])
      Zet
      print('TL results without FT\n')
      E1QS=np.cumsum(E1Q)
      DOLO=np.concatenate((DM[int(E1Q[1]*Coef):int(E1Q[1]),:],DM[int(E1Q[Class]):int(E1Q[Class+1]),:]))   
      Test=-torch.log10(D(torch.Tensor((DOLO).reshape(len(DOLO),1,S*int(W/2)))))
      Test[Test>Precision]=Precision
      Test=Test.detach().cpu().numpy()
      #plt.plot(Test)
      #plt.show()
      [FA,TA,FF]=PR(Test,Zet,1,KK)
      print('P= ',str(FA),'R= ',str(TA),'F1=',str(FF))

    if TL=="True":
      #print('TL results with FT\n')
      E1=np.cumsum(Zet)
      DMU=np.copy(DM)
      DMQ=np.copy(DM)
      E1Q=np.copy(E1)
      G=np.copy(Spec)
      DMMCopy=np.copy(DM)

      # FT, Algorith 1 --- Spectrum normalization using only the intact data%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
      for k in range(S):
        GA=G[k*int(W/2):(k+1)*int(W/2)];
        G1=np.mean(DMQ[0:int(E1Q[1]*Coef),k*int(W/2):(k+1)*int(W/2)]**2, axis=0)
        G1=G1/np.max(G1)
        L=np.argsort(GA)
        L1=np.argsort(G1)
        Kok=np.zeros((int(W/2),int(W/2)))
        for j in range(int(W/2)):
          Kok[j,:]=(np.abs(GA-G1[j]))
        K=np.argmin(Kok, axis=1)

      # Transfroming the rest of dataset suing mapping from the intact portion %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        for i in range(int(W/2)):
          DMMCopy[:,L[i]+k*int(W/2)]=np.copy(DMQ[:,L1[i]+k*int(W/2)])*np.sqrt(GA[int(L[i])]/G1[int(L1[i])])
      G111=(DMMCopy[int(E1Q[1]*Coef):int(E1Q[1]*(Coef+Coef2)),:])
      Test111=-torch.log10(D(torch.Tensor((G111).reshape(len(G111),1,S*int(W/2)))))
      TestAll=-torch.log10(D(torch.Tensor((DMMCopy).reshape(len(DMMCopy),1,S*int(W/2)))))
      mu, std = norm.fit(Test111.detach().cpu())
      KK=max(0.31,mu+3*std)
      print('DC'+str(Class)+' TL results with FT')
      E1QS=np.cumsum(E1Q)
      DOLO=np.concatenate((DMMCopy[int(E1Q[1]*(Coef+Coef2)):int(E1Q[1]),:],DMMCopy[int(E1Q[Class]):int(E1Q[Class+1]),:]))   
      Test=-torch.log10(D(torch.Tensor((DOLO).reshape(len(DOLO),1,S*int(W/2)))))
      Test[Test>Precision]=Precision
      Test=Test.detach().cpu().numpy()
      Zet=np.array([0,int(E1Q[1])-int(E1Q[1]*(Coef+Coef2)),E1Q[Class+1]-E1Q[Class]])
      print('SDD results with Damage class:\t',str(Class))
      plt.plot(Test)
      #plt.ylim([0,0.5])
      Zet_C1=Zet[1]
      plt.plot([0,Zet_C1+Zet[2]],[KK,KK],'r')
      plt.show()
      [PP[Class],RR[Class],FF[Class]]=PR2(Test,Zet,int(Zet_C1),KK)
      print('Precision= ',str(PP[Class]),'Recall= ',str(RR[Class]),'F1=',str(FF[Class]),'\n\n\n')

  plt.plot(PP)
  plt.plot(RR)
  plt.plot(FF)
  plt.xlim([1,CL])
  plt.title('Accuracy Metrics')
  plt.xlabel('Damage case',fontsize=24)
  plt.ylabel('PRF',fontsize=24)
  plt.legend(['Precision','Recall','F1'],fontsize=20)

In [ ]:
Binary_SDD(DMT,WS_D,NS_D,E1T,Data_CLT,D,Spec,Precision,TL=True)